<a href="https://colab.research.google.com/github/trDalmi/Data-Science-Portfolio/blob/main/Lane_detection_in_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2

In [2]:
from google.colab.patches import cv2_imshow

In [3]:
import numpy as np

In [5]:
def canny(img):
  gray =cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
  kernel = 5
  blur = cv2.GaussianBlur(gray,(kernel,kernel),0)
  canny = cv2.Canny(gray,50,150)
  return canny

In [6]:
def region_of_intererst(image):
  height = image.shape[0]
  polygons = np.array([[(200,height),(1100,height),(550,250)]])
  mask = np.zeros_like(image)           # full filled black sheet of img np.size
  cv2.fillPoly(mask,np.array(polygons),225)          # drawing a polygon on the filter with white infill.
  masked_image = cv2.bitwise_and(image,mask)    # Concatenating the mask with the original image using and operator
  return masked_image

In [9]:
def make_points(image,line):
  slope,intercept = line
  y1 = int(image.shape[0])
  y2 = int(y1*3.0/5)
  x1 = int((y1-intercept)/slope)
  x2 = int((y2-intercept)/slope)
  return [[x1,y1,x2,y2]]

In [13]:
def average_slope_intercept(image,lines):
  left_fit = []
  right_fit = []
  if lines is None:
    return None
  for line in lines:
    for x1,y1,x2,y2 in line:
      fit = np.polyfit((x1,x2),(y1,y2),1)
      slope = fit[0]
      intercept = fit[1]
      if slope < 0:
        left_fit.append((slope,intercept))
      else:
        right_fit.append((slope,intercept))
  left_fit_average = np.average(left_fit,axis=0)
  right_fit_average = np.average(right_fit,axis=0)
  left_line = make_points(image,left_fit_average)
  right_line = make_points(image,right_fit_average)
  averaged_lines = [left_line,right_line]
  return averaged_lines

In [14]:
def display_lines(img,lines):
  line_image = np.zeros_like(img)
  if lines is not None:
    for line in lines:
      for x1,y1,x2,y2 in line:
        cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
video_path = "/content/drive/MyDrive/data for cv/test2.mp4"
video = cv2.VideoCapture(video_path)

In [16]:
import time

In [30]:
while(video.isOpened()):
  _, frame = video.read()
  canny_image = canny(frame)
  cropped_image = region_of_intererst(canny_image)
  lines = cv2.HoughLinesP(cropped_image,2,np.pi/180, 100 ,np.array([]),minLineLength=40,maxLineGap=5)
  averaged_lines = average_slope_intercept(frame,lines)
  line_image = display_lines(frame,averaged_lines)
  combo_image = cv2.addWeighted(frame,0.8,line_image,1,1)
  cv2_imshow(combo_image)

  if cv2.waitKey(1) & 0xFF == ord('q'):  # Break the loop if 'q' is pressed
    break

video.release()


TypeError: cannot unpack non-iterable numpy.float64 object